# Getting Started (with Google Colab)
- In the [Google Colab](https://colab.research.google.com/) menu, go to File -> Open notebook
- In the dialogue that appears, click the GitHub tab
- Enter this repo as your GitHub URL, i.e., https://github.com/davedgd/studentcourse
- Click the search icon (or hit enter)
- Select the notebook you want to open to get started (e.g., studentcourse-llm.ipynb)
- **Note:** You should also change the runtime type to one with a GPU...
    - In the Google Colab menu, go to Runtime -> Change runtime type
    - Under 'Hardware accelerator' choose at least a 'T4 GPU' (or better)
    - Click the Save button
- To run the notebook, in the Google Colab menu, go to Runtime -> Run all
- If you see a 'Warning: This notebook was not authored by Google' popup, click the 'Run anyway' button
- Wait patiently, as it make take a few minutes to install the packages and run the notebook
- **Note:** You may see occasional error messages, particularly when installing/loading pip packages -- you can generally safely ignore these (as long as the notebook runs)

In [1]:
# install packages
!pip install transformers huggingface_hub -U -q --progress-bar off

# set environment variables
import os
os.environ['HF_XET_HIGH_PERFORMANCE'] = '1'

# import packages
from IPython.display import Markdown, display, clear_output
from transformers import pipeline, TextStreamer

In [2]:
# load model pipeline (adapted from https://qwen.readthedocs.io/en/latest/inference/transformers.html)
# notes:
# for a T4 GPU, consider recent, smaller models such as: Qwen/Qwen3-VL-4B-Instruct
# for newer architectures such as an L4, use: Qwen/Qwen3-VL-8B-Instruct-FP8

pipe = pipeline('text-generation',
                model = 'Qwen/Qwen3-VL-4B-Instruct',
                device_map = 'auto')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
The model 'Qwen3VLForConditionalGeneration' is not supported for text-generation. Supported models are ['PeftModelForCausalLM', 'ApertusForCausalLM', 'ArceeForCausalLM', 'AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BitNetForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'BltForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV2ForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'DogeForCausalLM', 'Dots1ForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'Ernie4_5ForCausalLM', 'Ernie4_5_MoeForCausalLM', 'Exaone4ForCausalLM', 'FalconForCausalLM', 'FalconH1ForCausalLM', 'FalconMambaForCausalLM', 'F

In [3]:
schema = '''
CREATE TABLE major (
	major VARCHAR(255),
	school VARCHAR(255),
	CONSTRAINT PK_major PRIMARY KEY (major)
	);

CREATE TABLE student (
	studentid INT,
	name VARCHAR(255),
	major VARCHAR(255),
	year INT,
	CONSTRAINT PK_student PRIMARY KEY (studentid),
	CONSTRAINT FK_student_to_major FOREIGN KEY (major)
		REFERENCES major(major)
	);

CREATE TABLE course (
	courseid INT,
	topic VARCHAR(255),
	title VARCHAR(255),
	CONSTRAINT PK_course PRIMARY KEY (courseid)
	);

CREATE TABLE takes (
	studentid INT,
	courseid INT,
	date date,
	CONSTRAINT PK_takes PRIMARY KEY (studentid, courseid),
	CONSTRAINT FK_takes_to_student FOREIGN KEY (studentid)
		REFERENCES student(studentid),
	CONSTRAINT FK_takes_to_course FOREIGN KEY (courseid)
		REFERENCES course(courseid)
	);

INSERT INTO major (major, school) VALUES ('COMM', 'Commerce');
INSERT INTO major (major, school) VALUES ('CS', 'Engineering');
INSERT INTO major (major, school) VALUES ('SE', 'Engineering');

INSERT INTO student VALUES (100, 'Lucy', 'COMM', 4);
INSERT INTO student VALUES (140, 'Brian', 'CS', 2);
INSERT INTO student VALUES (130, 'Lindsey', 'SE', 4);

INSERT INTO course VALUES (3200, 'RDBMS', 'Intro...');
INSERT INTO course VALUES (7450, 'Stats', 'Adv...');
INSERT INTO course VALUES (3600, 'Big Data', 'Intro...');
INSERT INTO course VALUES (3460, 'Finance', 'Intro...');
INSERT INTO course VALUES (4200, 'Global', 'Adv...');
INSERT INTO course VALUES (3100, 'Global', 'Intro...');

INSERT INTO takes VALUES (100, 3200, '2016-11-01');
INSERT INTO takes VALUES (100, 7450, '2016-11-01');
INSERT INTO takes VALUES (100, 4200, '2016-12-01');
INSERT INTO takes VALUES (140, 3600, '2016-12-01');
INSERT INTO takes VALUES (140, 3460, '2016-12-01');
INSERT INTO takes VALUES (140, 4200, '2016-01-01');
INSERT INTO takes VALUES (130, 3100, '2016-12-01');
INSERT INTO takes VALUES (130, 3600, '2016-12-01');
INSERT INTO takes VALUES (130, 3200, '2016-12-01');

CREATE TABLE engmajor (
	major VARCHAR(255),
	school VARCHAR(255),
	CONSTRAINT PK_engmajor PRIMARY KEY (major)
	);

INSERT INTO engmajor (major, school) VALUES ('CS', 'Engineering');
INSERT INTO engmajor (major, school) VALUES ('SE', 'Engineering');

CREATE TABLE students (
	studentid INT,
	name VARCHAR(255),
	major VARCHAR(255),
	year INT,
	CONSTRAINT PK_students PRIMARY KEY (studentid),
	CONSTRAINT FK_students_to_major FOREIGN KEY (major)
		REFERENCES major(major)
	);

CREATE TABLE studentemp (
	studentident INT,
	hourlysalary INT,
	CONSTRAINT PK_studentemp PRIMARY KEY (studentident)
	);

INSERT INTO studentemp VALUES (150, 12);
INSERT INTO studentemp VALUES (130, 15);
INSERT INTO studentemp VALUES (190, 20);

CREATE TABLE morestudent (
	studentid INT,
	name VARCHAR(255),
	major VARCHAR(255),
	year INT,
	CONSTRAINT PK_morestudent PRIMARY KEY (studentid, major),
	CONSTRAINT FK_morestudent_to_major FOREIGN KEY (major)
		REFERENCES major(major)
	);

INSERT INTO morestudent VALUES (150, 'Tim', 'COMM', 3);
INSERT INTO morestudent VALUES (140, 'Brian', 'CS', 2);
INSERT INTO morestudent VALUES (160, 'Will', 'COMM', 2);
'''

prompt = '''
### Task
Consider the database schema below and answer the question that follows.

### Database Schema
This query will run on a database whose schema is represented in this string:

```sql{schema}```

### SQL
Given the database schema, here is the SQL query that answers `{question}`:
'''

In [4]:
question = 'How many courses are each of the students in the STUDENT table currently taking?'

messages = [{'role': 'user', 'content': prompt.format(question = question, schema = schema)}]

streamer = TextStreamer(pipe.tokenizer, skip_prompt = True, skip_special_tokens = True)
messages = pipe(messages, max_new_tokens = 512, streamer = streamer)

clear_output(wait = True)
display(Markdown(messages[-1]['generated_text'][1]['content']))

To answer the question **“How many courses are each of the students in the STUDENT table currently taking?”**, we need to:

1. **Join the `student` table with the `takes` table** to link students with the courses they are taking.
2. **Group by student** to count the number of courses each student is taking.
3. **Select the student’s name and the count of courses**.

The `student` table contains student information (including `studentid` and `name`), and the `takes` table contains the courses each student is taking (via `studentid` and `courseid`). We can count the number of rows (courses) per student by grouping on `studentid` and then joining with the `student` table to get the name.

---

### ✅ Correct SQL Query:

```sql
SELECT 
    s.name,
    COUNT(t.courseid) AS num_courses
FROM 
    student s
JOIN 
    takes t ON s.studentid = t.studentid
GROUP BY 
    s.studentid, s.name;
```

---

### 💡 Explanation:

- `JOIN takes t ON s.studentid = t.studentid` → Links each student to their enrolled courses.
- `COUNT(t.courseid)` → Counts how many courses each student is taking.
- `GROUP BY s.studentid, s.name` → Groups the results by student to get the count per student.

---

### 📌 Note:

The `student` table has 3 students: `Lucy (100)`, `Brian (140)`, `Lindsey (130)`.  
We can verify the counts:

- **Lucy (100)**: Takes 3 courses → `RDBMS`, `Stats`, `Global` → ✅ 3
- **Brian (140)**: Takes 3 courses → `Big Data`, `Finance`, `Global` → ✅ 3
- **Lindsey (130)**: Takes 3 courses → `Global`, `Big Data`, `RDBMS` → ✅ 3

So the query returns:

| name    | num_courses |
|---------|-------------|
| Lucy    | 3           |
| Brian   | 3           |
| Lindsey | 3           |

---

✅ This query correctly answers the question.